In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
from langchain_groq import ChatGroq

In [11]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


llm=ChatGroq(model="gemma2-9b-it")
#llm = ChatOpenAI(model="gpt-4o")
result=llm.invoke("Hello")
result

AIMessage(content='Hello! 👋\n\nHow can I help you today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 10, 'total_tokens': 24, 'completion_time': 0.025454545, 'prompt_time': 0.001169269, 'queue_time': 0.188290984, 'total_time': 0.026623814}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6f8fa6ed-8ccf-4860-a84b-70cd9dbd0f18-0', usage_metadata={'input_tokens': 10, 'output_tokens': 14, 'total_tokens': 24})

In [16]:
from pydantic import BaseModel,Field

class EvaluationSchema(BaseModel):
    feedback: str = Field(description="Detailed feedback for the essay")
    score: int = Field(description="score out of 10",ge=0,le=10)

In [17]:
model = llm.with_structured_output(EvaluationSchema)

In [99]:
essay = """
Trees are an important part of nature. They are green, tall, and provide shade when it is sunny outside. Many people like trees because they look nice and make the air cleaner. Without trees, the world would be different, and not in a good way.

One reason trees are useful is that they give us oxygen. Humans and animals need oxygen to breathe, and trees help produce it. At the same time, trees take in carbon dioxide, which is something we don’t need too much of. Scientists say that too much carbon dioxide is bad for the environment, so trees help with that.

Another thing trees do is provide homes for animals. Birds build nests in them, and squirrels run up and down their branches. If there were no trees, these animals would have to find somewhere else to live, which might be difficult. Trees also give us fruit, like apples and oranges, which people enjoy eating.

Some people cut down trees to build houses or make paper. This is not always bad, but too much cutting can be a problem. If we keep cutting trees without planting new ones, there might not be enough left in the future. That’s why some organizations tell people to plant more trees.

In conclusion, trees are helpful in many ways. They give us oxygen, homes for animals, and even food. We should try to take care of them so they can keep doing these things. Maybe if everyone plants one tree, the world would be a better place.
"""

In [100]:
prompt = f"Evaluate the given essay and score it on basis of language quality, clearity of thought and provide detailed feedback and a score out of 10\n{essay}"
model.invoke(prompt)

EvaluationSchema(feedback='The essay provides a clear and well-structured explanation of the importance of trees. It covers various aspects, such as oxygen production, habitat provision, and the impact of deforestation. The language is simple and easy to understand. However, it could benefit from more specific examples and a deeper exploration of the complexities involved in deforestation and reforestation efforts.', score=7)

In [101]:
import operator
from typing import Annotated,TypedDict,List
from langgraph.graph import START,END, StateGraph

class ScoreState(TypedDict):

    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

In [102]:
# Declaring Nodes

def clarity(state: ScoreState):
    prompt = f"Evaluate the given essay and score it on basis of clarity of thought and score it out of 10\n{essay}"
    res = model.invoke(prompt)
    return {"clarity_feedback": res.feedback, "individual_scores": [res.score]}

def language(state: ScoreState):
    prompt = f"Evaluate the given essay and score it on basis of the language and grammer correctness and score it out of 10\n{essay}"
    res = model.invoke(prompt)
    return {"language_feedback": res.feedback,"individual_scores": [res.score]}

def analyis(state: ScoreState):
    prompt = f"Evaluate the given essay and score it on basis of the depth of analysis about the topic and score it out of 10\n{essay}"
    res = model.invoke(prompt)
    return {"analysis_feedback": res.feedback,"individual_scores": [res.score]}

def final_evaluation(state: ScoreState):
    prompt = f"Based on the following feedback on an the essay create a summarised feedback,language feedback : {state['language_feedback']} \n clarity of thought feedback {state['clarity_feedback']} \n depth of analysis feedback: {state['analysis_feedback']}"
    res = model.invoke(prompt)
    avg = sum(state["individual_scores"])/len(state["individual_scores"])
    return {"overall_feedback": res.feedback,"avg_score": [avg]}


In [103]:
graph = StateGraph(ScoreState)

graph.add_node("clarity",clarity)
graph.add_node("language",language)
graph.add_node("analysis",analyis)
graph.add_node("overall feedback",final_evaluation)

graph.add_edge(START,"clarity")
graph.add_edge(START,"language")
graph.add_edge(START,"analysis")

graph.add_edge("clarity","overall feedback")
graph.add_edge("language","overall feedback")
graph.add_edge("analysis","overall feedback")

graph.add_edge("overall feedback",END)

workflow =graph.compile()



In [104]:
intial_state = {"essay":essay}

In [105]:
workflow.invoke(intial_state)

{'essay': '\nTrees are an important part of nature. They are green, tall, and provide shade when it is sunny outside. Many people like trees because they look nice and make the air cleaner. Without trees, the world would be different, and not in a good way.\n\nOne reason trees are useful is that they give us oxygen. Humans and animals need oxygen to breathe, and trees help produce it. At the same time, trees take in carbon dioxide, which is something we don’t need too much of. Scientists say that too much carbon dioxide is bad for the environment, so trees help with that.\n\nAnother thing trees do is provide homes for animals. Birds build nests in them, and squirrels run up and down their branches. If there were no trees, these animals would have to find somewhere else to live, which might be difficult. Trees also give us fruit, like apples and oranges, which people enjoy eating.\n\nSome people cut down trees to build houses or make paper. This is not always bad, but too much cutting c